In [12]:
!pip install pybullet

In [0]:
import os
import random
import numpy as np
from collections import namedtuple,deque
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from gym import wrappers
import gym
import pybullet_envs




In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#Define Parameters

In [0]:
#Parameters from OpenAI Baselines
BUFFER_SIZE = int(1e6)   # replay buffer size
BATCH_SIZE = 100         # minibatch size
GAMMA = 0.99             # discount factor
TAU = 0.005              # for soft update of target parameters
LR_ACTOR = 1e-3          # learning rate of the actor 
LR_CRITIC = 1e-3         # learning rate of the critic
WEIGHT_DECAY = 0         # L2 weight decay
START_STEPS=1e4          # Number of steps for uniform-random action selection, before running real policy. Helps exploration.
NOISE_CLIP=0.5           # Limit for absolute value of target policy smoothing noise.
POLICY_DELAY=2           # Policy will only be updated once every policy_delay times for each update of the Q-networks
TARGET_NOISE=0.2         # Stddev for smoothing noise added to target policy.
ACT_NOISE=0.1            # Stddev for Gaussian exploration noise added to policy at training time. (At test time, no noise is added.)
MAX_EP_LEN=1000          # Maximum length of trajectory / episode / rollout.
MAX_NUM_EP =1000         # Maximum number of episodes
NUM_TEST_EP=10           # Number of episodes to test the deterministic policy at the end of episode
CLIP_NORM=False


#Define ReplayBuffer: network.py

In [0]:
class ReplayBuffer(object):
	def __init__(self, max_size=1e6):
		self.storage = []
		self.max_size = max_size
		self.ptr = 0

	def add(self, data):
		if len(self.storage) == self.max_size:
			self.storage[int(self.ptr)] = data
			self.ptr = (self.ptr + 1) % self.max_size
		else:
			self.storage.append(data)

	def sample(self, batch_size):
		ind = np.random.randint(0, len(self.storage), size=batch_size)
		x, y, u, r, d = [], [], [], [], []

		for i in ind: 
			X, Y, U, R, D = self.storage[i]
			x.append(np.array(X, copy=False))
			y.append(np.array(Y, copy=False))
			u.append(np.array(U, copy=False))
			r.append(np.array(R, copy=False))
			d.append(np.array(D, copy=False))

		return np.array(x), np.array(y), np.array(u), np.array(r).reshape(-1, 1), np.array(d).reshape(-1, 1)


#Define Actor-Critic Network: network.py

In [0]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, max_action, fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.max_action=max_action
        
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.bn1 = nn.BatchNorm1d(fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        if state.dim() == 1:
            state = torch.unsqueeze(state,0)
        a = F.relu(self.fc1(state))
        a = F.relu(self.fc2(a))
        return self.max_action*torch.tanh(self.fc3(a))
    
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)


class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        #Define 1st critic network
        self.critic1_fc1 = nn.Linear(state_size+action_size, fc1_units)
        self.critic1_bn1 = nn.BatchNorm1d(fc1_units)
        self.critic1_fc2 = nn.Linear(fc1_units, fc2_units)
        self.critic1_fc3 = nn.Linear(fc2_units, 1)
        
        #Define 2nd critic network
        self.critic2_fc1 = nn.Linear(state_size+action_size, fc1_units)
        self.critic2_bn1 = nn.BatchNorm1d(fc1_units)
        self.critic2_fc2 = nn.Linear(fc1_units, fc2_units)
        self.critic2_fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()
                       
       
    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        #Forward pass for critic 1
 
        xs = torch.cat([state, action], dim=1)
        x1 = F.relu(self.critic1_fc1(xs))
        x1 = F.relu(self.critic1_fc2(x1))
        q1= self.critic1_fc3(x1)
        #Forward pass for critic 2        
        x2 = F.relu(self.critic2_fc1(xs))
        x2 = F.relu(self.critic2_fc2(x2))  
        q2=self.critic2_fc3(x2)         
        return q1,q2
    
    def Q1(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        #Forward pass for critic 1   
        xs = torch.cat([state, action], dim=1)
        x1 = F.relu(self.critic1_fc1(xs))
        x1 = F.relu(self.critic1_fc2(x1))
        q1= self.critic1_fc3(x1)      
        return q1

    def reset_parameters(self):
        self.critic1_fc1.weight.data.uniform_(*hidden_init(self.critic1_fc1))
        self.critic1_fc2.weight.data.uniform_(*hidden_init(self.critic1_fc2))
        self.critic1_fc3.weight.data.uniform_(-3e-3, 3e-3)
    
        self.critic2_fc1.weight.data.uniform_(*hidden_init(self.critic2_fc1))
        self.critic2_fc2.weight.data.uniform_(*hidden_init(self.critic2_fc2))
        self.critic2_fc3.weight.data.uniform_(-3e-3, 3e-3)

#Define the agent: agent.py

In [0]:
class TD3Agent():
    """Interacts with and learns from the environment."""
       
    def __init__(self, state_size, action_size, min_action, max_action):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size
        self.total_steps=0
        self.max_action=max_action
        self.min_action=min_action

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, max_action).to(device)
        self.actor_target = Actor(state_size, action_size, max_action).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)
        self.hard_copy(self.actor_target,self.actor_local)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size).to(device)
        self.critic_target = Critic(state_size, action_size).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)
        self.hard_copy(self.critic_target,self.critic_local)
        # Freeze target networks with respect to optimizers (only update via polyak averaging       
        for param_critic,param_actor in zip(self.critic_target.parameters(),self.actor_target.parameters()):
           param_critic.requires_grad = False
           param_actor.requires_grad = False

        print('Created Actor-Critic Networks \n',self.actor_local,self.critic_local)      

    def act(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor_local(state).cpu().data.numpy().flatten() 

 

    def learn(self, memory, iterations, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        # states, actions, rewards, next_states, dones = experiences

        for it in range(iterations):

          # states, actions, rewards, next_states, dones=memory.sample()
          batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = memory.sample(BATCH_SIZE)
          states = torch.FloatTensor(batch_states).to(device)
          actions = torch.FloatTensor(batch_actions).to(device)
          next_states = torch.FloatTensor(batch_next_states).to(device)
          dones = torch.FloatTensor(1-batch_dones).to(device)
          rewards = torch.FloatTensor(batch_rewards).to(device)

          # ---------------------------- update critic ---------------------------- #
          #Get expected Q-values for (s,a)  for critic_local 1:Q1(s,a) and  critic_local 2:Q2(s,a)        
          Q1_expected,Q2_expected = self.critic_local(states, actions)
          # From the next state s', get next_action a'  usign actor_target
          #No need to call forward, when you call an object by itself(super(Actor)), the first method is executed
          actions_next = self.actor_target(next_states)
          #Define Gaussian noise and clamp it by the noiseclip parameter
          noise=torch.FloatTensor(batch_actions).data.normal_(0, TARGET_NOISE).to(device)
          noise=noise.clamp(-NOISE_CLIP,NOISE_CLIP)
          #Add Gaussian noise to the next action ,a', and clamp it in a range supported by the environement
          actions_next=(actions_next+noise).clamp(self.min_action,self.max_action)      
          #Get the Q-values for (s',a')  for critic 1:Qt1(s',a') and  critic2:Qt2(s',a')
          Q1_targets_next,Q2_targets_next = self.critic_target(next_states, actions_next)
          #Get the minimum of two Q-values min(Qt1,Qt2)
          Q_targets_next=torch.min(Q1_targets_next,Q2_targets_next)               
          # Compute Q targets for current states (y_i)
          y = rewards + (gamma * Q_targets_next * dones).detach()      
          # Compute critic loss:MSE_Loss(Q1(s,a),y)+MSE_Loss(Q2(s,a),y)
          critic_loss = F.mse_loss(Q1_expected, y)+F.mse_loss(Q2_expected, y)
          #initialize the optimizer
          self.critic_optimizer.zero_grad()
          critic_loss.backward()
          if CLIP_NORM is True:
              torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1) # clip gradient to max 1
          self.critic_optimizer.step()

          # ---------- update actor:using deterministic policy gradient(DPG) ------------------ #
          #Update actor only every POLICY_DELAY
          if  it % POLICY_DELAY == 0:  
            # Freeze critic-networks to reduce the computational effort computing gradients for them 
            for param_local,param_target in zip(self.critic_local.parameters(),self.critic_target.parameters()):
              param_local.requires_grad = False
              param_target.requires_grad = False

            actions_pred = self.actor_local(states)
            # Compute actor loss
            actor_loss = -self.critic_local.Q1(states, actions_pred).mean()
            # Minimize the loss
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()
            # Unfreeze critic-networks 
            for param_local,param_target in zip(self.critic_local.parameters(),self.critic_target.parameters()):
              param_local.requires_grad = True
              param_target.requires_grad = True
            # ----------------------- update target networks ----------------------- #
            with torch.no_grad():
              self.soft_update(self.critic_local, self.critic_target, TAU)
              self.soft_update(self.actor_local, self.actor_target, TAU)    

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

    def hard_copy(self,local_model,target_model):
        for target, local in zip(target_model.parameters(), local_model.parameters()):
            target.data.copy_(local.data)


    def save_checkpoints(self, filename):
        torch.save(self.critic_local.state_dict(), filename + "_critic")
        torch.save(self.critic_optimizer.state_dict(), filename + "_critic_optimizer")		
        torch.save(self.actor_local.state_dict(), filename + "_actor")
        torch.save(self.actor_optimizer.state_dict(), filename + "_actor_optimizer")
    
    def load_checkpoints(self, filename):
        self.critic_local.load_state_dict(torch.load(filename + "_critic"))
        self.actor_local.load_state_dict(torch.load(filename + "_actor"))



#Train the agent: main.py

In [0]:
def evaluate_policy(agent, eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    state = env.reset()
    done = False
    while not done:
      action = agent.act(state)
      state, reward, done, _ = env.step(action)
      avg_reward += reward
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("\n Policy rollout over the evaluation period: %f" % (avg_reward))
  print ("---------------------------------------")

  return avg_reward


In [0]:

def train(env, agent,file_name, max_episode_steps=1000,max_episodes=500, eval_episodes=10):
  # memory=ReplayBuffer( BUFFER_SIZE, BATCH_SIZE)
  memory=ReplayBuffer()
  episode_scores = deque(maxlen=eval_episodes)
  scores=[]
  total_timesteps = 0
  timesteps_since_eval = 0
  eval_freq=eval_episodes*max_episode_steps
  episode_num = 0
  done = True
  t0 = time.time()
  max_timesteps=max_episode_steps*max_episodes
  while total_timesteps < max_timesteps:    
    if done: 
      if total_timesteps != 0: 
        print("Total T: %d Episode Num: %d Episode T: %d Reward: %f  --  Wallclk T: %d sec" %  (total_timesteps, episode_num, episode_timesteps, episode_reward, int(time.time() - t0)))
        agent.learn(memory, episode_timesteps, GAMMA)      
        # Evaluate policy every t_eval and save
      if timesteps_since_eval >= eval_freq:
        timesteps_since_eval %= eval_freq
        evaluations.append(evaluate_policy(agent))
        agent.save_checkpoints(file_name)
        np.save("%s" % (file_name), evaluations)             
  
      # Reset environment
      state = env.reset()
      done = False
      episode_reward = 0
      episode_timesteps = 0
      episode_num += 1 
    
    # Select action randomly or according to policy
    if total_timesteps < START_STEPS:
      action = env.action_space.sample()
    else:
      action = agent.act(np.array(state))
      action = (action + np.random.normal(0, ACT_NOISE, size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)
    #Take action and observe reward and next state
    next_state,reward,done,_ = env.step(action) 
    done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
    episode_reward += reward
    #Store experience tuple (s,a,s',r) in replay memory 
    # memory.add(state, action, reward, next_state, done_bool)
    memory.add((state,next_state,action,reward,done_bool))
    state = next_state
    episode_timesteps += 1
    total_timesteps += 1
    timesteps_since_eval += 1
    episode_scores.append(episode_reward)    
    scores.append(episode_reward)   
    # if total_timesteps%50000==0:      
      # agent.save_checkpoints(file_name+'_episode_score_'+str(int(episode_reward)))
    # if np.mean(episode_scores)>2500 and total_timesteps>750000:
      # break
    
  # Final evaluation 
  evaluations.append(evaluate_policy(agent))
  agent.save_checkpoints(file_name+'final')
  np.save("%s" % (file_name+'final_eval'), evaluations) 
  return scores  

In [21]:
if not os.path.exists("./results"):
		os.makedirs("./results")
if  not os.path.exists("./videos"):
		os.makedirs("./videos")
seed=0
env_name = "HalfCheetahBulletEnv-v0" 
file_name= "./results/TD3"+ "%s" % (env_name) #Results
env = gym.make(env_name)  
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
#Get environment parameters
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
min_action=env.action_space.low[0]
max_action = env.action_space.high[0]
print('state_size,action_size',state_size,action_size)
print('min and max action=',min_action,max_action)

state_size,action_size 26 6
min and max action= -1.0 1.0


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [22]:
#Create an instance of  TD3Agent 
agent=TD3Agent(state_size, action_size, min_action, max_action)
evaluations = [evaluate_policy(agent)] 
#Train the agent
scores=train(env,agent,file_name,MAX_EP_LEN,MAX_NUM_EP,NUM_TEST_EP)
np.save("%s" % (file_name+'final_scores'), scores) 


Created Actor-Critic Networks 
 Actor(
  (fc1): Linear(in_features=26, out_features=400, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=6, bias=True)
) Critic(
  (critic1_fc1): Linear(in_features=32, out_features=400, bias=True)
  (critic1_bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (critic1_fc2): Linear(in_features=400, out_features=300, bias=True)
  (critic1_fc3): Linear(in_features=300, out_features=1, bias=True)
  (critic2_fc1): Linear(in_features=32, out_features=400, bias=True)
  (critic2_bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (critic2_fc2): Linear(in_features=400, out_features=300, bias=True)
  (critic2_fc3): Linear(in_features=300, out_features=1, bias=True)
)
---------------------------------------

 Policy rollout over the 

#Watch the agent

In [26]:
env_name = "HalfCheetahBulletEnv-v0"
seed = 0
file_name= "./results/TD3"+ "%s" % (env_name)+"final"
save_env_vid = True
env = gym.make(env_name)
max_episode_steps = 1000
if save_env_vid:
  env = wrappers.Monitor(env, './video', force = True)
  env.reset()
env.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
state_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
min_action=float(env.action_space.low[0])
agent=TD3Agent(state_size, action_size, min_action, max_action)
agent.load_checkpoints(file_name)
_ = evaluate_policy(agent, eval_episodes=NUM_TEST_EP)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Created Actor-Critic Networks 
 Actor(
  (fc1): Linear(in_features=26, out_features=400, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=6, bias=True)
) Critic(
  (critic1_fc1): Linear(in_features=32, out_features=400, bias=True)
  (critic1_bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (critic1_fc2): Linear(in_features=400, out_features=300, bias=True)
  (critic1_fc3): Linear(in_features=300, out_features=1, bias=True)
  (critic2_fc1): Linear(in_features=32, out_features=400, bias=True)
  (critic2_bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (critic2_fc2): Linear(in_features=400, out_features=300, bias=True)
  (critic2_fc3): Linear(in_features=300, out_features=1, bias=True)
)
---------------------------------------

 Policy rollout over the 

In [27]:
# !zip -r ./results/results.zip ./results
!zip -r  videos.zip ./video

  adding: video/ (stored 0%)
  adding: video/openaigym.video.2.120.video000008.mp4 (deflated 1%)
  adding: video/openaigym.video.2.120.video000001.mp4 (deflated 1%)
  adding: video/openaigym.video.2.120.video000000.meta.json (deflated 60%)
  adding: video/openaigym.video.2.120.video000001.meta.json (deflated 60%)
  adding: video/openaigym.video.2.120.video000000.mp4 (deflated 8%)
  adding: video/.ipynb_checkpoints/ (stored 0%)
  adding: video/openaigym.video.2.120.video000008.meta.json (deflated 60%)
